NUEVO ENFOQUE:  
Limitarse a mostrar datos de personas que hacen compras de impulso

# Trae librerías

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import json

# Viz
import plotly
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

# Google Auth
from oauth2client.service_account import ServiceAccountCredentials
from apiclient.discovery import build

# Google Trends
from pytrends.request import TrendReq

# Configura libreta

In [2]:
# px.defaults.template = 'ggplot2'

In [3]:
pd.options.display.max_rows = 150

# Utils

In [4]:
def initialize_analytics_reporting():
    """Instatiate GA UA Reporting API V4 client"""
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    service = build("analyticsreporting", "v4", credentials=credentials)
    return service


def metrics_dimensions_to_df(res):
    """Convert a non-empty response with metrics and dimensions to a dataframe."""
    report = res["reports"][0]
    dimensions = report["columnHeader"]["dimensions"]
    metrics = [
        m["name"] for m in report["columnHeader"]["metricHeader"]["metricHeaderEntries"]
    ]
    headers = [*dimensions, *metrics]

    data_rows = report["data"]["rows"]
    data = []
    for row in data_rows:
        data.append([*row["dimensions"], *row["metrics"][0]["values"]])

    return pd.DataFrame(data=data, columns=headers)


def metrics_to_df(res):
    """Convert a non-empty response with metrics and NO dimensions to a dataframe."""
    col_names = []
    for i in res["reports"][0]["columnHeader"]["metricHeader"]["metricHeaderEntries"]:
        col_names.append(i["name"])
    values = res["reports"][0]["data"]["rows"][0]["metrics"][0]["values"]

    data = (
        pd.Series(dict(zip(col_names, values)))
        .to_frame()
        .rename(columns={0: "metric_value"})
    )
    data = data.reset_index()
    data = data.rename(columns={"index": "metric_name"})
    return data

# Env

## <strong style="color: #0d4a42; opacity: .8;"> Vars

In [5]:
SCOPES = ["https://www.googleapis.com/auth/analytics.readonly"]
KEY_FILE_LOCATION = "proyecto-analytics-ets-e8e58fcb7c96.json"
VIEW_ID = "244974369"

## <strong style="color: #0d4a42; opacity: 0.8;"> Client

In [6]:
service = initialize_analytics_reporting()

# Tab 1: Conversión y visitas

## KPIs en indicadores

### Sesiones, transacciones, conversiones orgánicas

In [10]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
two_weeks_ago = a_week_ago - pd.Timedelta(days = 180)

# Parámetros generales del query
query_sampling = 'default'
query_size = '1000'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')
query_aux_date = two_weeks_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions',
                 'ga:transactions',
                 'ga:transactionsPerSession']

query_dimensions = ['ga:medium']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'filtersExpression': f'ga:medium==organic',
         'orderBys':[{'fieldName':'ga:transactions', 'sortOrder':'DESCENDING'}]
        }

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df.to_pickle('./datasets/organic_indicators.pkl')
df = pd.read_pickle('./datasets/organic_indicators.pkl')

In [11]:
df.head(3)

,medium,sessions,transactions,transactionsPerSession
0,organic,1790735,58606,3.272734


In [12]:
def make_indicator(data, data_delta, var, title):
    # Figura
    trace = go.Indicator(value = data.loc[:, var].sum(),
                         mode = 'number',
                         delta = {'reference': data_delta.loc[:, var].sum()*1.03})
    fig = go.Figure(data = trace)
    
    # Dimensiones
    fig.update_layout(width = 200,height = 200)

    # Estética
    fig.update_layout(plot_bgcolor='#F9F9F9',
                      paper_bgcolor='#F9F9F9',
                      title = title,
                      title_x = .5)
    # Fonts
    fig.update_traces(title_font_size = 25,
                      title_font_family = 'Open Sans',
                      number_font_size = 50,
                      number_font_family = 'Open Sans',
                      delta_font_size = 25,
                      delta_font_family = 'Open Sans',)
    
    return fig

In [ ]:
make_indicator(df, df, 'sessions', 'Organic sessions')

### Sesiones, transacciones, conversiones pagadas

In [14]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
query_utm = 'Mexico|in011'

# Parámetros generales del query
query_sampling = 'large'
query_size = '9999'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')
query_aux_date = two_weeks_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions',
                 'ga:transactions',
                 'ga:transactionsPerSession',
                ]

query_dimensions = ['ga:campaign']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'orderBys':[{'fieldName':'ga:transactions', 'sortOrder':'DESCENDING'}],
         'dimensionFilterClauses': [
             {'operator': 'OR',
              'filters': [
                        {'dimensionName': 'ga:campaign',
                         'operator': 'REGEXP',
                         'expressions': ['in011']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | Retargeting']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | General | En Línea']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT | Paper Edition Dates']}]}]}

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df.to_pickle('./datasets/paid_indicators.pkl')
df = pd.read_pickle('./datasets/paid_indicators.pkl')

In [15]:
df.head(10)

,campaign,sessions,transactions,transactionsPerSession
0,Mexico | TOEFL iBT,4493,71,1.580236
1,Mexico | TOEFL iBT | Paper Edition Dates,1747,45,2.575844
2,TOEFL iBT | General | En Línea,758,18,2.374670
3,TOEFL iBT | Retargeting,14877,0,0.000000
4,toefl-general-q122_1014-mex-ppc-in011-awa-mar2...,18,0,0.000000
5,toefl-ibt-b2c-q122_1014-mex-ppc-in011-awa-oct2...,45,0,0.000000
6,toefl-ibt-b2c-q122_1014-mex-ppc-in011-awa-oct2...,9,0,0.000000
7,toefl-ibt-b2c-q222_0106-mex-soc-in011-awa-ene2...,36,0,0.000000
8,toefl-ibt-b2c-q222_0120-mex-soc-in011-awa-ene2...,9,0,0.000000
9,toefl-ibt-b2c-q222_0127-mex-soc-in011-awa-ene2...,9,0,0.000000


In [16]:
make_indicator(df, df, 'sessions', 'Sesiones INCOM adquiridas')

## KPIs en tiempo

In [17]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)

# Parámetros generales del query
query_sampling = 'default'
query_size = '1000'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions']

query_dimensions = ['ga:date']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'filtersExpression': f'ga:medium==organic',
         'orderBys':[{'fieldName':'ga:sessions', 'sortOrder':'DESCENDING'}]
        }

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df.to_pickle('./datasets/organic_sessions_ts.pkl')
df = pd.read_pickle('./datasets/organic_sessions_ts.pkl')

In [18]:
df.head(3)

,date,sessions
0,2023-01-31,21642
1,2023-02-24,21378
2,2023-02-07,20835


In [20]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
query_utm = 'Mexico|in011'

# Parámetros generales del query
query_sampling = 'default'
query_size = '1000'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions']

query_dimensions = ['ga:date']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'orderBys':[{'fieldName':'ga:sessions', 'sortOrder':'DESCENDING'}],
         'dimensionFilterClauses': [
             {'operator': 'OR',
              'filters': [
                        {'dimensionName': 'ga:campaign',
                         'operator': 'REGEXP',
                         'expressions': ['in011']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | Retargeting']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | General | En Línea']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT | Paper Edition Dates']}]}]}

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df.to_pickle('./datasets/paid_sessions_ts.pkl')
df = pd.read_pickle('./datasets/paid_sessions_ts.pkl')

In [21]:
df.head(3)

,date,sessions
0,2023-02-20,891
1,2023-02-19,624
2,2023-02-18,517


In [22]:
df1 = pd.read_pickle('./datasets/organic_sessions_ts.pkl')
df2 = pd.read_pickle('./datasets/paid_sessions_ts.pkl')

In [23]:
def make_double_timeseries(data, data2, var, title):
    plot_data1 = data.resample(on = 'date', rule = 'w')[var].sum()
    plot_data2 = data2.resample(on = 'date', rule = 'w')[var].sum()
    
    # Create subplots with 2 rows and 1 column
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03)

    # Add traces for the first time series on the top subplot
    fig.add_trace(go.Scatter(x=plot_data1.index, y=plot_data1.values, mode = 'lines', name='Sesiones Orgánicas'), row=1, col=1)
    fig.update_yaxes(title_text='Sesiones', row=1, col=1)

    # Add traces for the second time series on the bottom subplot
    fig.add_trace(go.Scatter(x=plot_data2.index, y=plot_data2.values, mode = 'lines', name='Sesiones INCOM'), row=2, col=1)
    fig.update_yaxes(title_text='Sesiones', row=2, col=1)

    # Update the layout of the figure to improve the overall appearance
    fig.update_layout(width = 1000, height = 500)
    
    
    fig.update_layout(title = title,
                  title_x = .5,
                  plot_bgcolor='#F9F9F9',
                  paper_bgcolor='#F9F9F9',
                  showlegend = True,
                  hovermode = 'x unified')
    return fig

In [24]:
make_double_timeseries(df1, df2, 'sessions', 'Num de sesiones semanales')

# Desempeño de campañas

### Por fuente

In [25]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
two_weeks_ago = a_week_ago - pd.Timedelta(days = 180)
query_utm = 'Mexico|in011'

# Parámetros generales del query
query_sampling = 'small'
query_size = '1000'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')
query_aux_date = two_weeks_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions']

query_dimensions = ['ga:source',
                    'ga:medium',
                    'ga:campaign']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'orderBys':[{'fieldName':'ga:sessions', 'sortOrder':'DESCENDING'}],
         'dimensionFilterClauses': [
             {'operator': 'OR',
              'filters': [
                        {'dimensionName': 'ga:campaign',
                         'operator': 'REGEXP',
                         'expressions': ['in011']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | Retargeting']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | General | En Línea']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT | Paper Edition Dates']}]}]}

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df.to_pickle('./datasets/campaign_detail.pkl')
df = pd.read_pickle('./datasets/campaign_detail.pkl')

In [26]:
df.head(3)

,source,medium,campaign,sessions
0,google,cpc,TOEFL iBT | Retargeting,14222
1,google,cpc,Mexico | TOEFL iBT,4515
2,google,cpc,Mexico | TOEFL iBT | Paper Edition Dates,1570


In [27]:
def make_bars(data):
    plot_data = data.groupby(['source'])[['sessions']].sum().reset_index()
    plot_data = plot_data.sort_values(by = 'sessions')
    
    # Create plot
    fig = px.bar(plot_data,
                 y='source',
                 x='sessions',
                 orientation='h',
                 barmode='group')


    # Update the layout of the figure to improve the overall appearance
    fig.update_layout(width = 1000, height = 500)
    
    
    fig.update_layout(title = 'Sessions by campaign source',
                      title_x = .5,
                      plot_bgcolor='#F9F9F9',
                      paper_bgcolor='#F9F9F9')
    return fig

In [28]:
make_bars(df)

### Por tipo

In [29]:
def try_parse(text):
    try:
        return text.split('in011-')[1][:3]
    except:
        return text

In [30]:
df['type'] = df['campaign'].apply(lambda x: try_parse(x))

In [31]:
def make_bars2(data):
    plot_data = data.groupby(['type'])[['sessions']].sum().reset_index()
    plot_data = plot_data.sort_values(by = 'sessions')
    
    # Create plot
    fig = px.bar(plot_data,
                 y='type',
                 x='sessions',
                 orientation='h',
                 barmode='group')


    # Update the layout of the figure to improve the overall appearance
    fig.update_layout(width = 1000, height = 500)
    
    
    fig.update_layout(title = 'Sessions by campaign type',
                      title_x = .5,
                      plot_bgcolor='#F9F9F9',
                      paper_bgcolor='#F9F9F9')
    return fig

In [32]:
make_bars2(df)

### Visitas por campaña

In [33]:
def make_table(data):
    fig = go.Figure(data=[go.Table(header=dict(values=data.columns.tolist()),
                                   cells=dict(values=[data.campaign,
                                                      data.source,
                                                      data.medium,
                                                      data.sessions],
                                              align=['left','left','left','center']),
                                   columnwidth = [300,100,100])])

    # Customize the layout
    fig.update_layout(title='Table Example')

    # Remove the title
    fig.update_layout(title=None)

    # Adjust the margin and autosize properties
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        autosize=True
    )
    return fig

In [34]:
make_table(df[['campaign','source','medium','sessions']])

# Composición

## Composición por grupos y medios

In [35]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
two_weeks_ago = a_week_ago - pd.Timedelta(days = 180)
query_utm = 'Mexico|in011'

# Parámetros generales del query
query_sampling = 'small'
query_size = '1000'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')
query_aux_date = two_weeks_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions']

query_dimensions = ['ga:campaign', 'ga:channelGrouping']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'orderBys':[{'fieldName':'ga:sessions', 'sortOrder':'DESCENDING'}],
         'dimensionFilterClauses': [
             {'operator': 'OR',
              'filters': [
                        {'dimensionName': 'ga:campaign',
                         'operator': 'REGEXP',
                         'expressions': ['in011']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | Retargeting']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | General | En Línea']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT | Paper Edition Dates']}]}]}

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df.to_pickle('./datasets/paid_groupings.pkl')
df = pd.read_pickle('./datasets/paid_groupings.pkl')

In [36]:
df.head(3)

,campaign,channelGrouping,sessions
0,TOEFL iBT | Retargeting,Display,14222
1,Mexico | TOEFL iBT,Paid Search,4515
2,Mexico | TOEFL iBT | Paper Edition Dates,Paid Search,1570


In [37]:
def make_tree_paid_sources(data, title):
    plot_data = data[~(data['channelGrouping'].isin(['Direct','Organic Search', 'Referral']))]
    fig = px.treemap(plot_data,
                     path=[px.Constant("all"), 'channelGrouping', 'campaign'],
                     values='sessions',
                     title = title)
    
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(title = title,
                  width = 950,
                  height = 500,    
                  title_x = .5,
                  plot_bgcolor='#F9F9F9',
                  paper_bgcolor='#F9F9F9')
    
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    
    return fig

In [38]:
make_tree_paid_sources(df, 'Campaigns by Channel Grouping')

---

---

---

# Tab 2

Sobre información geográfica, demográfica y engagement

## Trend

In [39]:
pytrends = TrendReq(hl='en-US', tz=360)
search_term = ['TOEFL IBT', 'TOEFL', 'IBT', 'TOEFL TEST']
time_window = '2023-01-01 2023-05-01'
pytrends.build_payload(kw_list=search_term, timeframe=time_window, geo = 'MX')
interest_over_time_df = pytrends.interest_over_time()

interest_over_time_df.to_pickle('./datasets/search_trends.pkl')
interest_over_time_df = pd.read_pickle('./datasets/search_trends.pkl')

In [40]:
interest_over_time_df.head(3)

,TOEFL IBT,TOEFL,IBT,TOEFL TEST,isPartial
date,,,,,
2023-01-01,0,34,17,0,False
2023-01-02,8,43,0,0,False
2023-01-03,0,18,11,0,False


In [41]:
def make_timeseries(data, title):
    plot_data = interest_over_time_df.resample(rule = 'w').sum()
    # Add traces for the first time series on the top subplot
    fig = px.line(data_frame=plot_data, y = 'TOEFL IBT')
    
    # Update the layout of the figure to improve the overall appearance
    fig.update_layout(width = 1000, height = 500)
    
    
    fig.update_layout(title = title,
                  title_x = .5,
                  yaxis_title = 'Hits',
                  plot_bgcolor='#F9F9F9',
                  paper_bgcolor='#F9F9F9',
                  showlegend = True,
                  hovermode = 'x unified')
    return fig

In [42]:
make_timeseries(interest_over_time_df, 'Weekly search trend for: TOEFL IBT from México')

## Mapa

In [43]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
query_utm = 'Mexico|in011'

# Parámetros generales del query
query_sampling = 'large'
query_size = '9999'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')
query_aux_date = two_weeks_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions']

query_dimensions = ['ga:country', 'ga:region']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'orderBys':[{'fieldName':'ga:sessions', 'sortOrder':'DESCENDING'}],
         'dimensionFilterClauses': [
             {'operator': 'OR',
              'filters': [
                        {'dimensionName': 'ga:campaign',
                         'operator': 'REGEXP',
                         'expressions': ['in011']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | Retargeting']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | General | En Línea']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT | Paper Edition Dates']}]}]}

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df = df[df['country']=='Mexico'].reset_index(drop = True)
df = df[df['region']!='(not set)'].reset_index(drop = True)
mapa = {
    'Nuevo Leon': 'Nuevo León',
    'Michoacan': 'Michoacán',
    'San Luis Potosi': 'San Luis Potosí',
    'Mexico City': 'Ciudad de México',
    'State of Mexico': 'México',
    'Queretaro': 'Querétaro',
    'Yucatan': 'Yucatán'}

df['region'] = df['region'].replace(mapa)
df.to_pickle('./datasets/paid_map.pkl')
df = pd.read_pickle('./datasets/paid_map.pkl')

In [44]:
df.head(3)

,country,region,sessions
0,Mexico,Ciudad de México,3503
1,Mexico,Jalisco,1863
2,Mexico,México,1801


In [45]:
def make_choropleth(data):
    fig = px.choropleth(data,
                        geojson='mexico_estados.json',
                        locations='region',
                        featureidkey='properties.name',
                        color='sessions',
                        title='Sessions by Region',
                        color_continuous_scale='Viridis')

    fig.update_geos(showcountries=True,
                    showcoastlines=True,
                    fitbounds='locations')
    return fig

In [46]:
make_choropleth(df)

In [47]:
def make_bars3(data):
    plot_data = data.groupby(['region'])[['sessions']].sum().nlargest(5, columns = 'sessions').reset_index()
    plot_data = plot_data.sort_values(by = 'sessions')
    
    # Create plot
    fig = px.bar(plot_data,
                 y='region',
                 x='sessions',
                 orientation='h',
                 barmode='group')


    # Update the layout of the figure to improve the overall appearance
    fig.update_layout(width = 1000, height = 500)
    
    
    fig.update_layout(title = 'Sessions by Region (Top 5)',
                      title_x = .5,
                      plot_bgcolor='#F9F9F9',
                      paper_bgcolor='#F9F9F9')
    return fig

In [48]:
make_bars3(df)

## Segmentos

In [49]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
query_utm = 'Mexico|in011'

# Parámetros generales del query
query_sampling = 'large'
query_size = '9999'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')
query_aux_date = two_weeks_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions',
                 'ga:sessionDuration']

query_dimensions = ['ga:hour']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'orderBys':[{'fieldName':'ga:sessions', 'sortOrder':'DESCENDING'}],
         'dimensionFilterClauses': [
             {'operator': 'OR',
              'filters': [
                        {'dimensionName': 'ga:campaign',
                         'operator': 'REGEXP',
                         'expressions': ['in011']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | Retargeting']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | General | En Línea']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT | Paper Edition Dates']}]}]}

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

df.to_pickle('./datasets/paid_hours.pkl')
df = pd.read_pickle('./datasets/paid_hours.pkl')

In [50]:
def make_bars4(data):
    plot_data = data.set_index('hour').loc[['00','01','02',
                          '03','04','05',
                          '06','07','08',
                          '09','10','11',
                          '12','13','14',
                          '15','16','17',
                          '18','19','20',
                          '21','22','23'], :]
    
    # Create plot
    fig = px.bar(plot_data,
                 y='sessions',
                 barmode='group')


    # Update the layout of the figure to improve the overall appearance
    fig.update_layout(width = 1000, height = 500)
    
    
    fig.update_layout(title = 'Sessions by hour of day',
                      title_x = .5,
                      plot_bgcolor='#F9F9F9',
                      paper_bgcolor='#F9F9F9')
    return fig

In [51]:
make_bars4(df)

## Summary table

In [57]:
# Parámetros específicos del query
yesterday = pd.to_datetime('today') - pd.Timedelta(days = 1)
a_week_ago = yesterday - pd.Timedelta(days = 120)
query_utm = 'Mexico|in011'

# Parámetros generales del query
query_sampling = 'large'
query_size = '9999'

# Parámetros temporales del query
query_recent_date = yesterday.strftime('%Y-%m-%d')
query_old_date = a_week_ago.strftime('%Y-%m-%d')
query_aux_date = two_weeks_ago.strftime('%Y-%m-%d')


query_metrics = ['ga:sessions',
                 'ga:users','ga:adCost',
                 'ga:bounceRate',
                 'ga:avgSessionDuration',
                 'ga:transactions',
                 'ga:transactionRevenue']

query_dimensions = ['ga:campaign']

# Diccionario de query
query = {'viewId':VIEW_ID,
         'samplingLevel': query_sampling,
         'pageSize': query_size,
         'dateRanges':[{'startDate':query_old_date, 'endDate':query_recent_date}],
         'metrics':[{'expression':metric} for metric in query_metrics],
         'dimensions':[{'name': dimension} for dimension in query_dimensions],
         'orderBys':[{'fieldName':'ga:sessions', 'sortOrder':'DESCENDING'}],
         'dimensionFilterClauses': [
             {'operator': 'OR',
              'filters': [
                        {'dimensionName': 'ga:campaign',
                         'operator': 'REGEXP',
                         'expressions': ['in011']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | Retargeting']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['TOEFL iBT | General | En Línea']},
                        {'dimensionName': 'ga:campaign',
                         'operator': 'EXACT',
                         'expressions': ['Mexico | TOEFL iBT | Paper Edition Dates']}]}]}

# Query
response = service.reports().batchGet(body={'reportRequests':[query]}).execute()

# A dataframe
df = metrics_dimensions_to_df(response)

# A cols numéricas
for i in query_metrics:
    df[i] = pd.to_numeric(df[i])
    
for i in query_dimensions:
    if i == 'ga:date':
        df['ga:date'] = pd.to_datetime(df['ga:date'])
    else:
        pass

# Nombres razonables
new_cols = [x.split(':')[1] for x in df.columns]
df.columns = new_cols

numcols = df.select_dtypes('number').columns
df.loc[:, numcols] = df.loc[:, numcols].applymap('{:,.2f}'.format)

df.to_pickle('./datasets/summary_table.pkl')
df = pd.read_pickle('./datasets/summary_table.pkl')

In [58]:
df.head(3)

,campaign,sessions,users,adCost,bounceRate,avgSessionDuration,transactions,transactionRevenue
0,TOEFL iBT | Retargeting,"14,877.00","12,898.00",701.86,80.77,27.03,0.00,0.00
1,Mexico | TOEFL iBT,"4,493.00","2,781.00","2,497.08",34.72,259.23,71.00,"13,281.41"
2,Mexico | TOEFL iBT | Paper Edition Dates,"1,747.00","1,123.00","1,469.71",28.56,380.29,45.00,"8,245.17"


In [59]:
def make_table2(data):
    fig = go.Figure(data=[go.Table(header=dict(values=data.columns.tolist()),
                                   cells=dict(values=[data.campaign,
                                                      data.sessions,
                                                      data.users,
                                                      data.adCost,
                                                      data.bounceRate,
                                                      data.avgSessionDuration,
                                                      data.transactions,
                                                      data.transactionRevenue],
                                              align=['left','left','left','left',
                                                     'left','left','left','left']),
                                   columnwidth = [300,100,100])])

    # Customize the layout
    fig.update_layout(title='Table Example')

    # Remove the title
    fig.update_layout(title=None)

    # Adjust the margin and autosize properties
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        autosize=True
    )
    return fig

In [60]:
make_table2(df)